In [2]:
from field import F

# Generator
# G = F(85408008396924667383611388730472331217)
P = 17
G = F(3, P)

for i in range(0, P):
    print(i, G ** i)

G / 2

0 1
1 3
2 9
3 10
4 13
5 5
6 15
7 11
8 16
9 14
10 8
11 7
12 4
13 12
14 2
15 6
16 1


10

In [6]:
from polynomial import Polynomial
from field import F

P = 17
f = lambda x: F(x, P)
# f = lambda x: x
p = Polynomial([-1, 0, 8], f)
q = Polynomial([1,  1], f)

(p + q)([1, 2, 3])

[9, 0, 7]

In [6]:
import merkle

ls = ["A", "B", "C", "D", "E"]
hs = [merkle.hash_leaf(l) for l in ls]

for h in hs:
    print(h)

root = merkle.commit(hs)
print("root", root)

i = 2
proof = merkle.open(hs, i)
merkle.verify(proof, root, hs[i], i)

559aead08264d5795d3909718cdd05abd49572e84fe55590eef31a88a08fdffd
df7e70e5021544f4834bbee64a9e3789febc4be81470df629cad6ddb03320a5c
6b23c0d5f35d1b11f9b683f0b0a617355deb11277d91ae091d399c655b87940d
3f39d5c348e5b79d06e842c114e6cc571583bbf44e4b0ebfda1a01ec05745d43
a9f51566bd6705f7ea6ad54bb9deb449f795582d6529a0e22207b8981233ec58
root 2db1790243fe117685d21ed0ff5005d9832e5f32bf5b2b02cddf0f07a34421b2


True

In [3]:
import polynomial

p = polynomial.interp([1, 2], [1, 2])
p.degree()

1

In [2]:
# FRI domain
from field import find_generator, check_generator, get_primitive_root
from utils import is_prime

# Find L
n = 16
p = 337

assert is_prime(p), f'{p} is not prime'

# Find generator
g = find_generator(p)
assert g != None, "Generator not found"
print(f'generator g = {g}')

# Check order of the group generated by g
check_generator(g, p)

# Primitive Nth root
w = get_primitive_root(g, p, n)

# Evaluation domain L
L = [pow(w, i, p) for i in range(0, n)]
print(f'L = {L}')

generator g = 10
L = [1, 191, 85, 59, 148, 297, 111, 307, 336, 146, 252, 278, 189, 40, 226, 30]


In [3]:
import numpy as np
import random
import fri
from field import F
import polynomial
from polynomial import Polynomial
import merkle
import iop
from utils import is_pow2, is_prime

### Setup ###
# FRI domain L
# message (M) -> poly degree <= M - 1 -> RS code (N) <= |L| size of evaluation domain
# N = |L| = 2**K
# L = [1, w, w^2, ..., w^(N - 1)], w is Nth root of unity
# prime field F_p, |F_p| > |L|, N divides p - 1 (needed for finding primitive root of unity)

# Lenght of message
M = 8
ys = [random.randint(0, 1) for _ in range(M)]
print("ys:", ys)

# P = 1 + 407 * (1 << 119)
# Prime
P = 337
# Length of RS code
N = 16
# Nth primitive root
w = 191

# Expansion factor from message length M to RS code length N
# exp_factor * M = N
EXP_FACTOR = 2

# assert M < N < P
assert is_prime(P), f'{P} is not prime'
assert is_pow2(N), f'{N} is not a power of 2'

# TODO: plot
L = fri.domain(w, N, P)
p = polynomial.interp(L[:M], ys, lambda x: F(x, P))
print(f"Message length = {M}")
print(f"Polynomial degree = {p.degree()}")

### Commit ###
iop = iop.Writer()
prover = fri.Prover(
    N = N,
    P = P,
    w = w,
    poly = p,
    iop = iop,
    exp_factor = EXP_FACTOR
)

prover.commit()
print("codewords", prover.codewords)

### Query ###
verifier = fri.Verifier(
    N = N,
    P = P,
    w = w,
    challenges = iop.challenges,
    merkle_roots = iop.merkle_roots,
    exp_factor = EXP_FACTOR
)

# Random indexes without replacement
NUM_QUERIES = 2
assert NUM_QUERIES <= N

rands = [int(r) for r in np.random.choice(range(N), size=NUM_QUERIES, replace=False)]
for i in rands:
    (vals, proofs, codeword) = prover.prove(i)
    verifier.verify(i, vals, proofs, codeword)


ys: [0, 0, 1, 0, 1, 1, 0, 0]
Message length = 8
Polynomial degree = 7
codewords [[0, 0, 1, 0, 1, 1, 0, 0, 215, 295, 154, 253, 12, 89, 213, 295], [87, 154, 330, 200, 139, 167, 234, 269], [236, 33, 319, 185], [42, 42]]


In [2]:
# Polynomials

# 0 <= P(x) <= 9 integer 
# for all 1 <= x <= 1,000,000

# C(x) = x*(x-1)*...*(x-9)
# C(x) = 0 if 0 <= x <= 9
def c(x):
    v = 1
    for i in range(10):
        v *= (x - i)
    return v

# Z(x) = (x-1)*(x-2)*...*(x-1e6)
def z(x, n):
    v = 1
    for i in range(1, n + 1):
        v *= (x - i)
    return v

# C(P(x)) = 0 for 1 <= x <= 1,000,000
# C(P(x)) = Z(x)D(x)

# Polynomial commitment
# 1. Merkle tree of P(x) and D(x) at 1,000,000,000 points
# 2. Verifier randomly selects 16 values between 1 and 1,000,000,000
#    and asks the prover to provide the Merkle branches of P(x) and D(x)
# 3. Verifier checks
#    - Merkle proofs
#    - C(P(x)) = Z(x)D(x), P(x) and D(x) are provided in the Merkle proof

# Bivariate polynomial
# f(x) = polynomial with degree < 1,000,000
# Find g(x,y) such that g(x, x^1000) = f(x)

# 1. Polynomial commintment on g(x, y)
# for {(x, y^1000) for 1 <= x <= N and 1 <= y <= N}
# N = 1,000,000,000

# 2. Verifier picks randomly picks a few rows and columns
#    and for each row and column, asks for a few sample of points
#    one point in the sample is on the diagnol (x, x^1000)

# 3. Prover replies with does points and Merkle proofs

# TODO: why use bivariate polynomials?
# 4. - Verifier checks Merkle proofs
#    - Check polynomial corresponds to a low degree polynomial.
#      Degree less than number of samples requested.

In [7]:
from fft import fft_rec, fft, ifft, eval_poly
P = 17
N = 16
g = 3

# Multiplicative group
gs = [pow(g, i, P) for i in range(P - 1)]
assert len(set(gs)) == P - 1

# N-th primitive root
w = pow(g, (P - 1)//N, P)
print("w", w)
# Check w is a N-th primitive root
assert pow(w, N, P) == 1
assert pow(w, N//2, P) == -1 % P

# FFT evaluation domain
ws = [pow(w, i, P) for i in range(N)]
# Example polynomial
f = [i + 1 for i in range(N)]
print("f", f)
print("ws", ws)

e = eval_poly(f, ws, P)
r = fft_rec(f, ws, P)
l = fft(f, ws, P)
i = ifft(l, ws, P)

print("e", e)
print("r", r)
print("l", l)
print("i", i)
assert r == e
assert l == e
assert i == f

# Evaluate a polynomial on a domain larger than the degree of the polynomial
f = [i + 1 for i in range(N//2)] + ([0] * (N//2))
print("f", f)

ys = fft(f, ws, P)
print("ys", ys)

cs = ifft(ys, ws, P)
print("cs", cs)

assert cs == f

w 3
f [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
ws [1, 3, 9, 10, 13, 5, 15, 11, 16, 14, 8, 7, 4, 12, 2, 6]
e [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
r [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
l [0, 8, 2, 15, 7, 4, 6, 5, 9, 13, 12, 14, 11, 3, 16, 10]
i [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
f [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]
ys [2, 5, 1, 9, 12, 13, 3, 5, 13, 0, 6, 11, 14, 8, 8, 8]
cs [1, 2, 3, 4, 5, 6, 7, 8, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
# N-th root of unity
# Any number w such that w^N = 1 mod P

# Primitive N-th root of unity
# w^N = 1 and w^k != 1 for 0 < k < N 

# Steps to find the N-th primitive root of unity mod P
# 0. Pick a prime P -> This defines the finite field F[P]
#    TODO: Why prime?
#    - nonzero element has a multiplicative inverse
# 1. Consider the multiplicative group F[P]x = {1,2,3,...,P−1} 
# 2. Find a generator g of F[P]x
# 3. Find N = largest power of 2 dividing P−1
#    The subgroup of order N is:
#      H = { g^((P−1)/N)^i  for i = 0 ... N−1 }
# 4. To find a primitive N-th root of unity:
#    Let w = g^((P−1)/N) (g^(P-1) = 1 (Fermat's little theorem))
#    Verify:
#      - w^N = 1
#      TODO: why this check suffices
#      - w^(N/2) = -1

from field import find_generator
from utils import find_largest_pow_2

# 0. Pick a prime P
# Starkware prime
P = 2**251 + 17 * 2**192 + 1
# P = 17

# 2. Find a generator g
g = find_generator(P)
print("g = ", g)

# 3. Find N = largest power of 2 dividing P - 1
k = find_largest_pow_2(P - 1)
N = 2**k
print("k =", k)
print("n =", N)

w = pow(g, ((P - 1) // N), P)

assert pow(w, N, P) == 1
assert pow(w, N // 2, P) == (-1 % P)

print(f"{N} th primitive root of unity =", w)

g =  3
k = 192
n = 6277101735386680763835789423207666416102355444464034512896
6277101735386680763835789423207666416102355444464034512896 th primitive root of unity = 145784604816374866144131285430889962727208297722245411306711449302875041684


In [ ]:
# A Walk-Through of a Simple zk-STARK Proof

# N = 4
# A = [1, 0, 1, 1]
# g = 4 mod 17
# f(g[i]) = A[i] for i < N
# A[i] = 0 or 1 -> A[i](A[i] - 1) = 0 for i < N -> f(g[i])^2 - f(g[i]) = 0
# c(x) = f(x)^2 - f(x)
# u(x) = ((x-g[0])(x - g[1])...(x - g[N - 1])) = x^N - 1
# p(x) = c(x) / u(x)
# p(x) is polynomial of degree <= deg(c) - N <-> c(x) = 0 for x = g[i] <-> All A[i] = 0 or 1

# Query
# Verifier checks p(x)u(x) = c(x) (query x in L - G, query x in G breaks zero knowledge)
# 1. Verifier queries for z in L - G
# 2. Prover sends p(z) and f(z) and Merkle proofs of p(z) and f(z)
# 3. Verifier checks p(z)u(z) = c(z) and Merkle proofs

# num queries for high probability of fraud detection
# poly degree = d = 5
# evaluation domain L = 100
# num of dishonest evaluation = |L| - d = 95
# probability of getting caught after k queries = 1 - probability of not getting caught after k queries ~ 1 - (d / |L|)^k

# TODO - cheat if p is not a low degree
# TODO - example of false trace?

# Apply FRI to p

In [6]:
from fft import ifft

A = [1, 0, 1, 1]

P = 17
# N = len(A)
N = 4
g = 4
# Subgroup of order N generated by g
s = [pow(g, i, P) for i in range(N)]
assert len(set(s)) == N

ifft(A, s, P)

NameError: name 'N' is not defined